In [20]:
import numpy as np
import pandas as pd
#import training set
Train_df = pd.read_csv('/Users/weisihan/Downloads/we_data/train_newfeature.csv')
#import validation set
Valid_df = pd.read_csv('/Users/weisihan/Downloads/we_data/valid_newfeature.csv')

In [22]:
# bids which click =1
df1 = Train_df[Train_df['click']==1]
# bids which click =0
df0 = Train_df[Train_df['click']==0]
# get 5% of click =0 data, down sampling
df2=df0.sample(frac=0.1)

In [23]:
# combine 10% of click =0 data and all click=1, down sampling
combined_df = pd.concat([df1, df2],axis=0)
combined_df.head()
#combined_df.shape
#(244712, 127)

,click,slotwidth,slotheight,slotprice,android,ios,linux,mac,other,windows,...,slotformat_Na,advertiser_1458,advertiser_2259,advertiser_2261,advertiser_2821,advertiser_2997,advertiser_3358,advertiser_3386,advertiser_3427,advertiser_3476
1104,1,1000,90,80,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4949,1,336,280,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
8878,1,300,250,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9790,1,320,50,160,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
10000,1,300,250,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [24]:
#combined_df.to_csv('mytrainingset.csv', encoding='utf-8', index=False)

In [25]:
combined_df.shape

(244712, 127)

In [26]:
# columns of without click
cols=list(combined_df.columns)[1:]

In [27]:
train = combined_df
train_y = train['click']
train_x = train[cols]

第一步：确定学习速率和tree_based 参数调优的估计器数目

树的最大深度一般3-10 
max_depth = 5 
节点分裂所需的最小损失函数下降值0.1到0.2 
gamma = 0 
采样 
subsample= 0.8, 
colsample_bytree = 0.8 
比较小的值，适用极不平衡的分类问题 
min_child_weight = 1 
类别十分不平衡 
scale_pos_weight = 1

In [28]:
from xgboost import XGBClassifier
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

第二步： max_depth 和 min_weight 参数调优

grid_search参考 
http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html 
http://blog.csdn.net/abcjennifer/article/details/23884761 
网格搜索scoring=’roc_auc’只支持二分类，多分类需要修改scoring(默认支持多分类)

In [33]:
#13:06
"""param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}"""

"""param_test1 = {
 'max_depth':[3,5,7,9],
 'min_child_weight':range[1,3,5]
}"""


param_test2 = {
 'max_depth':[6],
 'min_child_weight':[8,9]}


from sklearn import svm, grid_search, datasets
from sklearn import grid_search

gsearch1 = grid_search.GridSearchCV(
estimator = XGBClassifier(
learning_rate =0.1,
n_estimators=100, max_depth=5,
min_child_weight=1,
gamma=0,
subsample=0.8,
colsample_bytree=0.8,
objective= 'binary:logistic',
nthread=4,
scale_pos_weight=1,
seed=27),
param_grid = param_test2,
scoring='roc_auc',
n_jobs=4,
iid=False,cv=5)
gsearch1.fit(train_x,train_y )
gsearch1.grid_scores_, gsearch1.best_params_,gsearch1.best_score_
#网格搜索scoring='roc_auc'只支持二分类，多分类需要修改scoring(默认支持多分类)

([mean: 0.87137, std: 0.00480, params: {'max_depth': 6, 'min_child_weight': 8},
  mean: 0.87070, std: 0.00449, params: {'max_depth': 6, 'min_child_weight': 9}],
 {'max_depth': 6, 'min_child_weight': 8},
 0.8713662625399625)

第三步：gamma参数调优

In [34]:
#13:36
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = grid_search.GridSearchCV(
estimator = XGBClassifier( 
learning_rate =0.1, 
n_estimators=100, 
max_depth=6, 
min_child_weight=7, 
gamma=0, 
subsample=0.8, 
colsample_bytree=0.8, 
objective= 'binary:logistic', 
nthread=4, 
scale_pos_weight=1,
seed=27), 
param_grid = param_test3, 
scoring='roc_auc',
n_jobs=4,
iid=False, 
cv=5)
gsearch3.fit(train_x,train_y)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

([mean: 0.87141, std: 0.00571, params: {'gamma': 0.0},
  mean: 0.86976, std: 0.00369, params: {'gamma': 0.1},
  mean: 0.86940, std: 0.00541, params: {'gamma': 0.2},
  mean: 0.87032, std: 0.00425, params: {'gamma': 0.3},
  mean: 0.86870, std: 0.00399, params: {'gamma': 0.4}],
 {'gamma': 0.0},
 0.8714083735434602)

第四步：调整subsample 和 colsample_bytree 参数

In [35]:
#13:48
#取0.6,0.7,0.8,0.9作为起始值
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}

gsearch4 = grid_search.GridSearchCV(
estimator = XGBClassifier(
learning_rate =0.1,
n_estimators=100,
max_depth=6,
min_child_weight=7,
gamma=0,
subsample=0.8,
colsample_bytree=0.8,
objective= 'binary:logistic',
nthread=4,
scale_pos_weight=1,
seed=27),
param_grid = param_test4,
scoring='roc_auc',
n_jobs=4,
iid=False,
cv=5)
gsearch4.fit(train_x,train_y)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

([mean: 0.86791, std: 0.00288, params: {'colsample_bytree': 0.6, 'subsample': 0.6},
  mean: 0.86820, std: 0.00350, params: {'colsample_bytree': 0.6, 'subsample': 0.7},
  mean: 0.86969, std: 0.00337, params: {'colsample_bytree': 0.6, 'subsample': 0.8},
  mean: 0.86977, std: 0.00458, params: {'colsample_bytree': 0.6, 'subsample': 0.9},
  mean: 0.86696, std: 0.00493, params: {'colsample_bytree': 0.7, 'subsample': 0.6},
  mean: 0.86987, std: 0.00423, params: {'colsample_bytree': 0.7, 'subsample': 0.7},
  mean: 0.87040, std: 0.00269, params: {'colsample_bytree': 0.7, 'subsample': 0.8},
  mean: 0.86991, std: 0.00520, params: {'colsample_bytree': 0.7, 'subsample': 0.9},
  mean: 0.86834, std: 0.00314, params: {'colsample_bytree': 0.8, 'subsample': 0.6},
  mean: 0.86790, std: 0.00389, params: {'colsample_bytree': 0.8, 'subsample': 0.7},
  mean: 0.87141, std: 0.00571, params: {'colsample_bytree': 0.8, 'subsample': 0.8},
  mean: 0.87039, std: 0.00387, params: {'colsample_bytree': 0.8, 'subsample'

第五步：正则化参数调优

In [36]:
param_test6 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = grid_search.GridSearchCV(
estimator = XGBClassifier(
learning_rate =0.1,
n_estimators=100,
max_depth=6,
min_child_weight=7,
gamma=0,
subsample=0.8,
colsample_bytree=0.8,
objective= 'binary:logistic',
nthread=4,
scale_pos_weight=1,
seed=27),
param_grid = param_test6,
scoring='roc_auc',
n_jobs=4,
iid=False,
cv=5)
gsearch6.fit(train_x,train_y)
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

([mean: 0.87141, std: 0.00571, params: {'reg_alpha': 1e-05},
  mean: 0.86952, std: 0.00523, params: {'reg_alpha': 0.01},
  mean: 0.87101, std: 0.00494, params: {'reg_alpha': 0.1},
  mean: 0.86972, std: 0.00569, params: {'reg_alpha': 1},
  mean: 0.82934, std: 0.00461, params: {'reg_alpha': 100}],
 {'reg_alpha': 1e-05},
 0.871408321878795)

第六步：降低学习速率

In [ ]:
xgb4 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=100,
 max_depth=6,
 min_child_weight=7,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 reg_alpha=0.01,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb4, train, predictors)